<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab05/expansao_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import nltk
import re
import collections
import heapq
import time
import bisect
import math
import random
from nltk.tokenize import RegexpTokenizer

ranking = pd.read_csv("https://raw.githubusercontent.com/adautofbn/ri_labs/master/lab04/ranking.csv") # Ranking adquirido na tarefa anterior

In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]

### 1. Calcule as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com as 4 métricas que vimos na aula. Você deve produzir uma tabela similar à tabela 6.3 do capítulo 6 do livro texto (pág. 204). Qual métrica você acha que obteve os melhores resultados? Por que?

In [0]:
def calc_mim(query, word):
  return (random.random() * 100, word[0])
  
def calc_emim(query, word):
  return (random.random() * 100, word[0])
  
def calc_x(query, word):
  return (random.random() * 100, word[0])
  
def calc_dice(query, word):
  return (random.random() * 100, word[0])

In [0]:
def create_metrics_table(query):
  top_10_table = pd.DataFrame()
  MIM = []
  EMIM = []
  X = []
  Dice = []
  
  for row in ranking.itertuples():
    word = [row[1],row[2]]
    if word[0] != query:
      bisect.insort(MIM, calc_mim(query,word))
      bisect.insort(EMIM,calc_emim(query, word))
      bisect.insort(X,calc_x(query, word))
      bisect.insort(Dice,calc_dice(query, word))

  MIM.reverse()
  EMIM.reverse()
  X.reverse()
  Dice.reverse()
  
  top_10_table['MIM'] = [word[1] for word in MIM]
  top_10_table['EMIM'] = [word[1] for word in EMIM]
  top_10_table['X²'] = [word[1] for word in X]
  top_10_table['Dice'] = [word[1] for word in Dice]
  
  return top_10_table.head(10)

In [0]:
table_1 = create_metrics_table(queries[0])
table_2 = create_metrics_table(queries[1])
table_3 = create_metrics_table(queries[2])
table_4 = create_metrics_table(queries[3])
table_5 = create_metrics_table(queries[4])

####Tabela para a consulta: 'Bolsonaro'

In [152]:
table_1.index += 1
table_1

,MIM,EMIM,X²,Dice
1,tratado,descartou,internos,bandeirantes
2,desidratado,questionado,instância,consideração
3,fizessem,alain,destinação,tereza
4,dobra,desagradando,cujo,nimo
5,informalidade,seguir,lançado,necessária
6,redes,maiores,documental,anos
7,peres,extra,monetário,reich
8,populares,recortadas,ditar,brasília
9,caymmi,nome,índia,nunca
10,ranking,isenção,parcela,compreenda


####Tabela para a consulta: 'Educação'

In [153]:
table_2.index += 1
table_2

,MIM,EMIM,X²,Dice
1,comentou,tropa,faziam,reitera
2,podem,alguma,passando,cerca
3,montando,melhorar,globo,declaração
4,deus,causa,jogado,civil
5,funcef,vêm,pública,sai
6,petrobrás,alimentando,encerrada,positivo
7,aposentadorias,liberação,pedem,pontual
8,claro,declarou,impossível,gaza
9,correios,pondera,opção,hahahahha
10,enfrentamento,mente,chico,realizada


####Tabela para a consulta: 'Ministério'

In [154]:
table_3.index += 1
table_3

,MIM,EMIM,X²,Dice
1,funcionariam,menor,ganhou,resgatar
2,adversária,convulsão,rio,conferência
3,agronegócio,fundamental,resnais,volte
4,gilvam,lvii,constrangido,extras
5,vantagens,russos,lembrar,christchurch
6,episódio,inteiro,parar,legislativa
7,trabalhadores,convocação,explicou,lugar
8,regime,congresso,reunidos,críticos
9,tomar,solo,pesado,etapa
10,espino,independência,próxima,confiança


####Tabela para a consulta: 'Brasil'

In [155]:
table_4.index += 1
table_4

,MIM,EMIM,X²,Dice
1,resultados,supostamente,ignoram,torturadores
2,maduro,dessa,discurso,rendimentos
3,ativistas,concedidas,receba,legislado
4,chama,milícias,distribuição,surge
5,interfax,fraco,permissão,fonte
6,pontual,ministros,pontos,excederam
7,menores,serviçal,tramitação,entendo
8,aves,acordo,ódio,compartilha
9,diálogo,reginaldo,vanucchi,inep
10,referir,cristalizou,national,exibição


####Tabela para a consulta: 'Economia'

In [156]:
table_5.index += 1
table_5

,MIM,EMIM,X²,Dice
1,empregos,possam,postando,atingir
2,prender,teixeira,entanto,acompanhada
3,preparar,advertir,institucional,inclusive
4,realizada,cabide,detalhes,indústria
5,atirar,passageiro,assessorar,justificativa
6,trazendo,ver,atualmente,questionou
7,ódio,realizado,soja,fictícios
8,tensões,mercados,commodities,defensoria
9,cidades,compartilham,ignácio,international
10,enfatizou,presenças,portas,mulheres


### 2. De acordo com a métrica que deu os melhores resultados na sua opinião, execute agora cada consulta (usando a abordagem documento- ou termo-por-vez)  expandido-a com: os top-3, top-5 e top-10 documentos. O que acontece com a precisão dos resultados em cada caso? Aumenta ou diminui? Justifique bem sua resposta.

In [0]:
def doc_a_time(query, indexes, k):
    start = time.time()
    
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for doc in range(1, len(result.text)+1):
      doc_score = 0
      for index in query_indexes:
          for i in index:
              if i[0] == doc:
                  doc_score += i[1]
                  break
      if doc_score != 0:
        heapq.heappush(rank, (doc_score, doc))
    
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:
def term_a_time(query, indexes, k):
    start = time.time()
    
    a = {} # Accumulator HashTable
    query_indexes = [] # Array
    rank = [] # HeapQueue, implementation of PriorityQueue
    
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    end = time.time() - start
    return heapq.nlargest(k, rank), end